# YOLOs

## Download data

In [1]:
# Delete all folders:
!rm -r ./*

In [2]:
data = 1
version = 3

!mkdir data
!mkdir data/images
!mkdir data/annotations

if data == 1:
  !curl -L "https://public.roboflow.com/ds/SEdhxhUOcM?key=caE3egJHqp" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  !cp test/*.jpg data/images
  !cp test/*.xml data/annotations
  !cp train/*.jpg data/images
  !cp train/*.xml data/annotations

  !rm -r train
  !rm -r test
  !rm README.dataset.txt
  !rm README.roboflow.txt

if data == 2:
  from google_drive_downloader import GoogleDriveDownloader as gdd

  drive_id = '1qrdZlaDi272eA79b0uCwwqPrm2Q_WI3k'
  gdd.download_file_from_google_drive(file_id=drive_id,
                                      dest_path='/content/data.zip',
                                      unzip=True)
  !rm /content/data.zip

  !mv NEU-DET/IMAGES/* data/images/
  !mv NEU-DET/ANNOTATIONS/* data/annotations/
  !rm -r NEU-DET

A streamkimeneten csak az utolsó 5000 sor látható.
 extracting: train/000135_jpg.rf.0ca2fda801cbc26acad48a4970d5fd9c.xml  
 extracting: train/001254_jpg.rf.0cb559668c7f79bc74c9f6b1a276d90d.xml  
 extracting: train/002215_jpg.rf.0c7007e87377d919cb34a6b62325380a.xml  
 extracting: train/002296_jpg.rf.0c9e2f0756e58251ba576c4799a52013.xml  
 extracting: train/002055_jpg.rf.0d0d8e31527f9ca4331834bd0923016e.xml  
 extracting: train/000888_jpg.rf.0c1eb6f457b9f7f371ddc6eaed1fc4de.xml  
 extracting: train/001653_jpg.rf.0cc3a72fd244aa7aadee6c4d766190df.xml  
 extracting: train/004482_jpg.rf.0d541b29bd4b4ededc19b5cafebb4fe9.xml  
 extracting: train/004320_jpg.rf.0cedfd83d5055b3a601e711b7e1c149c.xml  
 extracting: train/004334_jpg.rf.0c7c46e2c74725ad2f93e916dc2422dd.xml  
 extracting: train/000735_jpg.rf.0d5b96188e544511c13ac1e1bfff0b87.xml  
 extracting: train/002336_jpg.rf.0ca4ec6f6ebaabe8df1781814ea731d9.xml  
 extracting: train/003070_jpg.rf.0d4f1aa374695130ec80a94eb7d3029f.xml  
 extracting: 

## Convert labels to YOLO format (=.txt)

In [3]:
from xml.dom import minidom
import os
import glob


lut={}
if data == 1:
  lut["head"] = 0
  lut["helmet"] = 1
  lut["person"] = 2

if data == 2:
  lut["crazing"] = 0
  lut["inclusion"] = 1
  lut["patches"] = 2
  lut["pitted_surface"] = 3
  lut["rolled-in_scale"] = 4
  lut["scratches"] = 5


def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_xml2yolo(path):
    for fname in glob.glob(path + "*.xml"):      
        xmldoc = minidom.parse(fname)
        fname_out = (fname[:-4]+'.txt')
        with open(fname_out, "w") as f:

            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)

            for item in itemlist:
                # get class label
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in lut:
                    label_str = str(lut[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table" % classid)

                # get bbox coordinates
                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert_coordinates((width,height), b)
                #print(bb)

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')

        print ("wrote %s" % fname_out)

In [4]:
convert_xml2yolo('data/annotations/')

A streamkimeneten csak az utolsó 5000 sor látható.
wrote data/annotations/004140_jpg.rf.15799dbe567b00289e44bc3087ad4e67.txt
wrote data/annotations/005519_jpg.rf.9fc377450890006672a2ed4f0991c3b8.txt
wrote data/annotations/004080_jpg.rf.59efbf01c9ef05ba0b16cc0c95571dc9.txt
wrote data/annotations/002019_jpg.rf.b92988855104852df0088edba38d4ce3.txt
wrote data/annotations/000467_jpg.rf.09244a3e97312e8868d3a132cb22f926.txt
wrote data/annotations/002573_jpg.rf.09899b9befc5eba133753cd6396a9db3.txt
wrote data/annotations/006475_jpg.rf.24ea8b8c78a6e3c22d33d8c84fc96f3d.txt
wrote data/annotations/002522_jpg.rf.990c83a38d6f345ce1d63f222c4caa1f.txt
wrote data/annotations/004886_jpg.rf.e2fec3553feeaa2d8f658eade829f96e.txt
wrote data/annotations/003107_jpg.rf.46644fdec9d5b9060c4669db880d0044.txt
wrote data/annotations/002491_jpg.rf.ffd04888dd70532ff3289eea7d96a336.txt
wrote data/annotations/001631_jpg.rf.cbfd37a89536436389f93b36d5336ea9.txt
wrote data/annotations/006331_jpg.rf.8f648d5ce859b0e299b4ee9f

## Split data to 3 sets and save it in YOLO-organized directories

In [5]:
import random
import glob

if data == 1:
  classes = ["head", "helmet", "person"]
if data == 2:
  classes = ['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']

train_names = []
valid_names = []
test_names = []

train_perc = 70
valid_perc = 15
test_perc = 15

image_num = len(glob.glob("data/images/*"))

if data == 1:
  train_num = int(image_num * train_perc/100)
  valid_num = int(image_num * valid_perc/100)
  test_num = int(image_num * test_perc/100)

  files = glob.glob("data/images/*")
  random.shuffle(files)
  valid_data = files[:valid_num]
  test_data = files[valid_num:(valid_num + test_num)]
  train_data = files[(valid_num + test_num):]
  train_names += train_data
  valid_names += valid_data
  test_names += test_data

if data == 2:
  train_num = int(image_num/len(classes) * train_perc/100)
  valid_num = int(image_num/len(classes) * valid_perc/100)
  test_num = int(image_num/len(classes) * test_perc/100)

  for c in classes:
    files = glob.glob("data/images/" + c + "*")
    random.shuffle(files)
    train_data = files[:train_num]
    valid_data = files[train_num:(train_num + valid_num)]
    test_data = files[(train_num + valid_num):]
    train_names += train_data
    valid_names += valid_data
    test_names += test_data

print('Training images:', len(train_names))
print('Validation images:', len(valid_names))
print('Test images:', len(test_names))

Training images: 4925
Validation images: 1055
Test images: 1055


In [6]:
from shutil import copyfile

# Copy image files based on the shuffled split:
!mkdir data_yolo5/
!mkdir data_yolo5/train/
!mkdir data_yolo5/train/images
!mkdir data_yolo5/train/labels

!mkdir data_yolo5/valid/
!mkdir data_yolo5/valid/images
!mkdir data_yolo5/valid/labels

!mkdir data_yolo5/test/
!mkdir data_yolo5/test/images
!mkdir data_yolo5/test/labels

for name in train_names:
  copyfile(name, 'data_yolo5/train/images/' + name.split('/')[-1])
for name in valid_names:
  copyfile(name, 'data_yolo5/valid/images/' + name.split('/')[-1])
for name in test_names:
  copyfile(name, 'data_yolo5/test/images/' + name.split('/')[-1])


# Copy label files based on the shuffled split:
for name in train_names:
  copyfile(name.replace('.jpg', '.txt').replace('images', 'annotations'), 'data_yolo5/train/labels/' + name.split('/')[-1].replace('.jpg', '.txt'))
for name in valid_names:
  copyfile(name.replace('.jpg', '.txt').replace('images', 'annotations'), 'data_yolo5/valid/labels/' + name.split('/')[-1].replace('.jpg', '.txt'))
for name in test_names:
  copyfile(name.replace('.jpg', '.txt').replace('images', 'annotations'), 'data_yolo5/test/labels/' + name.split('/')[-1].replace('.jpg', '.txt'))

## Create .yaml file for YOLOv5 Pytorch training

In [7]:
content = 'train: ../train/images/\n' + \
          'val: ../valid/images/\n\n' + \
          'nc: ' + str(len(classes)) + '\n\n' + \
          'names: ' + str(classes)

with open('data_yolo5/data.yaml', 'w') as file:
  file.write(content)

In [8]:
!zip -r data{version}_yolo5.zip data_yolo5/

A streamkimeneten csak az utolsó 5000 sor látható.
  adding: data_yolo5/train/images/003656_jpg.rf.71243360c196ccc5317d662d3ed0faee.jpg (deflated 1%)
  adding: data_yolo5/train/images/002107_jpg.rf.bb44ae1fae44646599ca9e37dd09a339.jpg (deflated 1%)
  adding: data_yolo5/train/images/005849_jpg.rf.85c3ed66e23f81cc542b316f6530132c.jpg (deflated 1%)
  adding: data_yolo5/train/images/006054_jpg.rf.c0ce057657f5e484bc1426803fed3586.jpg (deflated 0%)
  adding: data_yolo5/train/images/002827_jpg.rf.541b9045b13a975e92cba8ee013b7d15.jpg (deflated 0%)
  adding: data_yolo5/train/images/005460_jpg.rf.4a72b0c265b8cb6478dd393f9c65c1a0.jpg (deflated 0%)
  adding: data_yolo5/train/images/003348_jpg.rf.4c88376f4aa9b6cce7258c2111c978df.jpg (deflated 0%)
  adding: data_yolo5/train/images/004695_jpg.rf.7712715be75f56bfb835e70fcf6e1f3e.jpg (deflated 0%)
  adding: data_yolo5/train/images/006397_jpg.rf.52dd3c5e27eafc0668558982045987be.jpg (deflated 1%)
  adding: data_yolo5/train/images/006728_jpg.rf.95e738ca70

## Create file system for YOLOv4

In [9]:
# Create directories
!mkdir data_yolo4
!mkdir data_yolo4/train
!mkdir data_yolo4/valid
!mkdir data_yolo4/test

In [10]:
# Copy contents
!cp data_yolo5/train/images/* data_yolo4/train
!cp data_yolo5/valid/images/* data_yolo4/valid
!cp data_yolo5/test/images/* data_yolo4/test

!cp data_yolo5/train/labels/* data_yolo4/train
!cp data_yolo5/valid/labels/* data_yolo4/valid
!cp data_yolo5/test/labels/* data_yolo4/test

In [11]:
# Create label file

with open('data_yolo4/train/_darknet.labels', 'w') as file:
  for item in classes[:-1]:
    file.write("%s\n" % item)
  file.write("%s" % classes[-1])

!cp data_yolo4/train/_darknet.labels data_yolo4/valid/
!cp data_yolo4/train/_darknet.labels data_yolo4/test/

In [12]:
!zip -r data{version}_yolo4.zip data_yolo4/

A streamkimeneten csak az utolsó 5000 sor látható.
  adding: data_yolo4/train/001978_jpg.rf.6e7eaed8e37fdf797160dcee7f3fdfc5.txt (deflated 51%)
  adding: data_yolo4/train/000677_jpg.rf.7c3658c297bc9af12f63fb8ebc0969ab.txt (deflated 16%)
  adding: data_yolo4/train/002222_jpg.rf.d80a720e897c332a328a185b743a6442.txt (deflated 16%)
  adding: data_yolo4/train/001944_jpg.rf.b79d06b8010cbfa485c2f6cd0cd490b3.txt (deflated 16%)
  adding: data_yolo4/train/003318_jpg.rf.ab087fdec59b425efcc4f8b21f86b786.txt (deflated 49%)
  adding: data_yolo4/train/001917_jpg.rf.5851f055538ca7e9b56abf963ad4fbdd.txt (deflated 13%)
  adding: data_yolo4/train/005272_jpg.rf.63d54ab53c82edb85ef3f411d7fa7c44.txt (deflated 65%)
  adding: data_yolo4/train/002628_jpg.rf.9b6c8d9668abb087bacb6f4bda89442f.txt (deflated 56%)
  adding: data_yolo4/train/002292_jpg.rf.46c61ce89f866d84884988ce3499597c.jpg (deflated 1%)
  adding: data_yolo4/train/006381_jpg.rf.a250cbea836013b2535fc86b5e60038a.txt (deflated 30%)
  adding: data_yolo4

# TFRecords

In [13]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [14]:
!git clone --quiet https://github.com/tensorflow/models.git
!pip install tf_slim
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py
%cd /content/

     |████████████████████████████████| 358kB 4.9MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-

In [15]:
# Create folder with temporary folder to store the split data
!git clone https://github.com/douglasrizzo/detection_util_scripts
!mkdir data_tfrecord
!mkdir data_tfrecord/temp
!mkdir data_tfrecord/temp/train
!mkdir data_tfrecord/temp/valid
!mkdir data_tfrecord/temp/test

Cloning into 'detection_util_scripts'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 785 (delta 27), reused 27 (delta 9), pack-reused 733
Receiving objects: 100% (785/785), 48.04 MiB | 33.91 MiB/s, done.
Resolving deltas: 100% (484/484), done.


In [16]:
# Copy temporary data in the appropriate folders based on the splits
for name in train_names:
  copyfile(name.replace('.jpg', '.xml').replace('images', 'annotations'), 'data_tfrecord/temp/train/' + name.split('/')[-1].replace('.jpg', '.xml'))
for name in valid_names:
  copyfile(name.replace('.jpg', '.xml').replace('images', 'annotations'), 'data_tfrecord/temp/valid/' + name.split('/')[-1].replace('.jpg', '.xml'))
for name in test_names:
  copyfile(name.replace('.jpg', '.xml').replace('images', 'annotations'), 'data_tfrecord/temp/test/' + name.split('/')[-1].replace('.jpg', '.xml'))

In [17]:
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET
from tqdm import tqdm


def __list_to_csv(annotations, output_file):
   column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
   xml_df = pd.DataFrame(annotations, columns=column_name)
   xml_df.to_csv(output_file, index=None)


def xml_to_csv(xml_dir, output_file):
   """Reads all XML files, generated by labelImg, from a directory and generates a single CSV file"""
   annotations = []
   for xml_file in tqdm(glob.glob(xml_dir + '/*.xml')):
      tree = ET.parse(xml_file)
      root = tree.getroot()
      for member in root.findall('object'):
         value = (root.find('filename').text,
                  int(root.find('size').find('width').text),
                  int(root.find('size').find('height').text),
                  member.find('name').text,
                  int(member.find('bndbox').find('xmin').text),
                  int(member.find('bndbox').find('ymin').text),
                  int(member.find('bndbox').find('xmax').text),
                  int(member.find('bndbox').find('ymax').text))
         annotations.append(value)

   __list_to_csv(annotations, output_file)

In [18]:
# Create CSV files for each split (overwriting mistake from the original scrip above)
xml_to_csv('data_tfrecord/temp/train/', 'data_tfrecord/annotations_train.csv')
xml_to_csv('data_tfrecord/temp/valid/', 'data_tfrecord/annotations_valid.csv')
xml_to_csv('data_tfrecord/temp/test/', 'data_tfrecord/annotations_test.csv')


# Delete the temporary folder
!rm -r data_tfrecord/temp

100%|██████████| 1055/1055 [00:00<00:00, 9653.77it/s]


In [19]:
# Check and correct wrong names in the CSV files (sometimes .jpg is missing):
import pandas as pd

for csv_name in ['train', 'valid', 'test']:
  df = pd.read_csv('data_tfrecord/annotations_' + csv_name + '.csv')
  if len(df.loc[~df["filename"].str.contains('.jpg')]) > 0:
    print(len(df.loc[~df["filename"].str.contains('.jpg')]), "fixed names without jpg format in", csv_name)
    df.loc[~df["filename"].str.contains('.jpg'), "filename"] = df.loc[~df["filename"].str.contains('.jpg'), "filename"] + ".jpg"
    df.to_csv('data_tfrecord/annotations_' + csv_name + '.csv')

In [20]:
!python detection_util_scripts/generate_pbtxt.py csv data_tfrecord/annotations_train.csv data_tfrecord/label_map.pbtxt

In [21]:
!python detection_util_scripts/generate_tfrecord.py data_tfrecord/annotations_train.csv data_tfrecord/label_map.pbtxt data/images data_tfrecord/data_train.tfrecord
!python detection_util_scripts/generate_tfrecord.py data_tfrecord/annotations_valid.csv data_tfrecord/label_map.pbtxt data/images data_tfrecord/data_valid.tfrecord
!python detection_util_scripts/generate_tfrecord.py data_tfrecord/annotations_test.csv data_tfrecord/label_map.pbtxt data/images data_tfrecord/data_test.tfrecord

groups: 100% 4925/4925 [00:06<00:00, 730.09it/s]
Successfully created the TFRecords: /content/data_tfrecord/data_train.tfrecord
groups: 100% 1055/1055 [00:01<00:00, 731.29it/s]
Successfully created the TFRecords: /content/data_tfrecord/data_valid.tfrecord
groups: 100% 1055/1055 [00:01<00:00, 792.29it/s]
Successfully created the TFRecords: /content/data_tfrecord/data_test.tfrecord


In [22]:
!zip -r data{version}_tfrecord.zip data_tfrecord/

  adding: data_tfrecord/ (stored 0%)
  adding: data_tfrecord/data_test.tfrecord (deflated 2%)
  adding: data_tfrecord/data_valid.tfrecord (deflated 2%)
  adding: data_tfrecord/annotations_test.csv (deflated 79%)
  adding: data_tfrecord/data_train.tfrecord (deflated 2%)
  adding: data_tfrecord/label_map.pbtxt (deflated 53%)
  adding: data_tfrecord/annotations_valid.csv (deflated 79%)
  adding: data_tfrecord/annotations_train.csv (deflated 79%)


In [23]:
!zip -r hardhat_test_images.zip data_yolo4/test/*

  adding: data_yolo4/test/000018_jpg.rf.aa084e30d2b7507b19fc60cb45f9c62d.jpg (deflated 1%)
  adding: data_yolo4/test/000018_jpg.rf.aa084e30d2b7507b19fc60cb45f9c62d.txt (deflated 16%)
  adding: data_yolo4/test/000020_jpg.rf.ff66b8deb18fbf371555a6ca94dd8854.jpg (deflated 1%)
  adding: data_yolo4/test/000020_jpg.rf.ff66b8deb18fbf371555a6ca94dd8854.txt (deflated 32%)
  adding: data_yolo4/test/000024_jpg.rf.f4dbede31bcaa30e4264e464b2d32266.jpg (deflated 0%)
  adding: data_yolo4/test/000024_jpg.rf.f4dbede31bcaa30e4264e464b2d32266.txt (deflated 60%)
  adding: data_yolo4/test/000040_jpg.rf.ab00c59a7d1431a6920b1f9ba21166c7.jpg (deflated 0%)
  adding: data_yolo4/test/000040_jpg.rf.ab00c59a7d1431a6920b1f9ba21166c7.txt (deflated 50%)
  adding: data_yolo4/test/000045_jpg.rf.f8429e05f94a05b82b3bf7256063c313.jpg (deflated 0%)
  adding: data_yolo4/test/000045_jpg.rf.f8429e05f94a05b82b3bf7256063c313.txt (deflated 34%)
  adding: data_yolo4/test/000046_jpg.rf.0e35b2bc17b8775dce91c9d84353530e.jpg (deflate